In [3]:
from crewai import  Agent, Task, Crew
from youtube_transcript_api import YouTubeTranscriptApi
import spacy
from nltk.tokenize import sent_tokenize
from transformers import pipeline


In [15]:
from crewai_tools import Tool


/Users/jasroopsingh/Desktop/yt_2_blog/yt_blog/lib/python3.13/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


ImportError: cannot import name 'Tool' from 'crewai_tools' (/Users/jasroopsingh/Desktop/yt_2_blog/yt_blog/lib/python3.13/site-packages/crewai_tools/__init__.py)

In [4]:
def extract_transcript_details(youtube_video_url):
    try:
        video_id=youtube_video_url.split("=")[1]
        
        transcript_text=YouTubeTranscriptApi.get_transcript(video_id)

        transcript = ""
        for i in transcript_text:
            transcript += " " + i["text"]

        return transcript

    except Exception as e:
        raise e

In [5]:
transcript=extract_transcript_details("https://www.youtube.com/watch?v=84pxFTPsVkY")
print(transcript)

 I have with me Joe Root after this third test match at Lords. Another hard fought five days in this series. How are you feeling about this win? Uh obviously very excited, very a little bit of relief in there as well. Um another brilliant test match you know I think this series is going to be like that and we you know we still got two more to go. So um yeah it's um it's really pleasing to be on the the right side of the result today. Credit to India the way that they played um took it very deep. We knew it was it was um it was going to be close coming into today, but I think the way that the guys stuck at it there were, you know, the first session in particular, the way that they bowled um taking crucial wickets early um getting ahead of the game and then the way Ben managed it towards the end was was incredible really um and amazing for for um show to get that moment at the end having broken his hand within this game. Um so have to go on and face Jasper Bummer and um and the rest of t

In [6]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize



[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1018)>


In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1018)>


False

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [18]:
from dotenv import load_dotenv
load_dotenv()
import os


In [ ]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                           verbose=True,
                           temperature=0.5,
                           google_api_key=os.getenv("GOOGLE_API_KEY"))




In [7]:
nlp = spacy.load("en_core_web_sm")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use mps:0


In [12]:

def summarize_large_text(text, max_chunk_words=600):
    """
    Preprocesses, chunks, and summarizes long text using spaCy and BART.
    
    Args:
        text (str): Raw input text (e.g. YouTube transcript).
        max_chunk_words (int): Maximum words per chunk (default 600).
    
    Returns:
        str: Final combined summary.
    """
    # Step 1: Preprocess and sentence split using spaCy
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 5]

    # Step 2: Chunk sentences based on word count
    chunks, current_chunk = [], []

    for sentence in sentences:
        current_len = sum(len(s.split()) for s in current_chunk)
        if current_len + len(sentence.split()) < max_chunk_words:
            current_chunk.append(sentence)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    # Step 3: Summarize each chunk
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
        summaries.append(summary)

    # Step 4: Combine summaries
    final_summary = " ".join(summaries)
    return final_summary


In [13]:
summary=summarize_large_text(transcript)
print(summary)

England captain Joe Root talks to Sportsmail after the third test at Lords. Root: "Another brilliant test match you know I think this series is going to be like that" Root's 37th test 100 and eighth at the Lord's cricket ground. I've not quite given my best throughout this series as yet. nice to to get, you know, get in the series if you like and um and make a valuable contribution which has led to a great result for us. a tough contested battle always between you and Jasp Bumra. 11 dismissals now.
